In [36]:
import os
import os.path
from mycode.find_files_with_extension import find_file_with_extension as find_ext
import datetime

Columns:

Cruise Station Type mon/day/yr hh:mm Lon (°E) Lat (°N) Bot. Depth [m] Depth [m] QF Temperature [°C] QF Salinity [PSU] QF

In [63]:
class StationMetadataList:
    """Station metadata class"""
    def __init__(self):
        self.dict = {}  # dict full of other dicts
        
    def populate(self, path):
        with open(path, 'r') as metatable_f:
            file_content = metatable_f.read().splitlines()
            
        station_field = None
        number_field = None
        latitude_north_field = None
        longitude_east_field = None
        datetime_field = None
        serial_number_field = None
        filename_field = None
        bottom_depth_field = None

        for idx, field in enumerate(file_content[0].split('\t')):
            lwd_field = field.lower()

            if 'station' in lwd_field:
                station_field = idx
            elif 'num' in lwd_field:
                number_field = idx
            elif 'latn' in lwd_field:
                latitude_north_field = idx
            elif 'lone' in lwd_field:
                longitude_east_field = idx
            elif 'datetime' in lwd_field:
                datetime_field = idx
            elif 'sn' in lwd_field:
                serial_number_field = idx
            elif 'fname' in lwd_field:
                filename_field = idx
            elif 'botdepth' in lwd_field:
                bottom_depth_field = idx
            else:
                print(f'Strange field: {field} {idx}')

        for line in file_content[1:]:
            line_content = line.split('\t')
            dt_string = line_content[datetime_field]
            dt_format = '%d-%m-%YT%H:%M:%S'
            dt_object = datetime.datetime.strptime(dt_string, dt_format)
            
            dict_key = f'{line_content[number_field]:0>3}-{line_content[station_field]}-{line_content[filename_field]}-{line_content[serial_number_field]}'
            self.dict[dict_key] = {'station' : line_content[station_field],
                                 'num' : f'{int(line_content[number_field]):0>3}',
                                 'latn' : f'{float(line_content[latitude_north_field]):.8f}',
                                 'lone' : f'{float(line_content[longitude_east_field]):.8f}',
                                 'datetime' : line_content[datetime_field],
                                 'date' : line_content[datetime_field][0:10],
                                 'time' : line_content[datetime_field][11:],
                                 'sn' : line_content[serial_number_field],
                                 'fname' : line_content[filename_field],
                                 'botdepth' : f'{float(line_content[bottom_depth_field]):.2f}',
                                 'odv_date' : f'{dt_object.strftime("%m/%d/%y")}',
                                 'odv_time' : f'{dt_object.strftime("%H:%M:%S")}'}
# dt_string = "2020-12-18 3:11:09" 
# format = "%Y-%m-%d %H:%M:%S:%s"
# dt_object = datetime.datetime.strptime(dt_string, format)
class SVP_Station:
    """A class for handling SVP station"""
    
    def __init__(self):
        self.metadata = None
        self.idxs = []
        self.pressure = []
        self.temperature = []
        self.sound_speed = []
        
    def load_from_valeport_minisvp(self, path, metatable_object):
        with open(path, 'r') as minisvp_f:
            file_content = minisvp_f.read().splitlines()
            
        file_name = os.path.splitext(os.path.basename(path))[0]
        sn = None
        
        # read header first
        for line in file_content[0:9]:
            if 'MiniSVP' in line:
                sn = line.split(' ')[2].strip()
        
        # read the data
        enum = 0
        for line in file_content[9:]:
            line_content = line.split('\t')
            if line_content[2] == '0000.000':
                pass
            else:
                
                self.idxs.append(enum)
                self.pressure.append(float(line_content[0]))
                self.temperature.append(float(line_content[1]))
                self.sound_speed.append(float(line_content[2]))
                enum += 1
        e = 0
        for key in metatable_object.dict.keys():
            if file_name == metatable_object.dict[key]['fname'] and sn == metatable_object.dict[key]['sn']:
                self.metadata = metatable_object.dict[key].copy()
                self.metadata['instrument'] = 'Valeport MiniSVP'
                break
                
    def save_to_odv_spreadsheet(self, cruise_string, output):
        # Type mon/day/yr hh:mm Lon (°E) Lat (°N) Bot. Depth [m] Depth [m] QF Temperature [°C] QF Salinity [PSU] QF
        output_file_name = f'ODV_{self.metadata["num"]}_{self.metadata["station"]}.txt'
        with open(os.path.join(output,output_file_name), 'w') as odv_f:
            odv_f.write(f'Cruise\tStation\tType\tmon/day/yr\thh:mm\tLon (°E)\tLat (°N)\tBot. Depth [m]\tPressure [dbar]\tQF\tTemperature [°C]\tQF\tSoundSpeed [m/s]\tQF\n')
            for i, idx in enumerate(self.idxs):
                odv_f.write(f'{cruise_string}\t{self.metadata["station"]}\tC\t'+
                              f'{self.metadata["odv_date"]}\t{self.metadata["odv_time"]}\t'+
                              f'{self.metadata["lone"]}\t{self.metadata["latn"]}\t'+
                              f'{self.metadata["botdepth"]}\t{self.pressure[i]}\t1\t'+
                              f'{self.temperature[i]}\t1\t{self.sound_speed[i]}\t1\n')
                
    def save_to_esimo_csv_format(self, esimo_num, cruise_string, output):
        output_file_name = f'E{esimo_num}O2_{cruise_string}_H14_{self.metadata["num"]}.csv'
        with open(os.path.join(output,output_file_name), 'w', encoding='Windows-1251') as esimo_f:
            esimo_f.write(f'rec_num;identific;station;SN;Pres;Temp;SoundVel\n')
            for i, idx in enumerate(self.idxs):
                esimo_f.write(f"{idx};{self.metadata['num'].lstrip('0')};{self.metadata['station']};{self.metadata['sn']};"+
                              f"{self.pressure[i]};{self.temperature[i]};{self.sound_speed[i]}\n")
            
    

In [64]:
path = r'D:\aa_MyExpeditions\Cr_2022-06-01_ABP49\aa_Hydrology_and_SVP\OceanDataView\SVP\75749\V000025_1_BlueDeep.TXT'
metatable = r'D:\aa_MyExpeditions\Cr_2022-06-01_ABP49\aa_Hydrology_and_SVP\OceanDataView\ABP49_SVP_Metadata_List.txt'
output = r'D:\aa_MyExpeditions\Cr_2022-06-01_ABP49\aa_Hydrology_and_SVP\OceanDataView'

df = r'D:\aa_MyExpeditions\Cr_2022-06-01_ABP49\aa_Hydrology_and_SVP\OceanDataView\SVP'
esimo_num = '090005'
cruise_string = 'ABP_49'

svp_paths = find_ext('.TXT', df)
# for idx, path in enumerate(svp_paths):
#     print(f'{idx}   {path}')

metadata_list = StationMetadataList()
metadata_list.populate(metatable)


for svp_path in svp_paths:
    station = SVP_Station()
    station.load_from_valeport_minisvp(svp_path, metadata_list)
    station.metadata['platform'] = '"R/V "Akademik Boris Petrov"'
    # print(station.metadata)
    # station.save_to_esimo_csv_format(esimo_num, cruise_string, output)
    station.save_to_odv_spreadsheet(cruise_string, output)

# station = SVP_Station()
# station.load_from_valeport_minisvp(path, metadata_list)
# station.metadata['platform'] = '"R/V "Akademik Boris Petrov"'

# station.metadata
# esimo_num = '090005'
# cruise_string = 'ABP_49'
# station.save_to_esimo_csv_format(esimo_num, cruise_string, output)

Searching *.TXT files in directory:D:\aa_MyExpeditions\Cr_2022-06-01_ABP49\aa_Hydrology_and_SVP\OceanDataView\SVP


In [57]:
metadata_list.dict

{'001-49001-V000210-75748': {'station': '49001',
  'num': '001',
  'latn': '54.90311167',
  'lone': '19.77374500',
  'datetime': '01-06-2022T15:24:00',
  'date': '01-06-2022',
  'time': '15:24:00',
  'sn': '75748',
  'fname': 'V000210',
  'botdepth': '46.00',
  'odv_datetime': '2022-06-01T15:24:00'},
 '002-49002-V000213-75748': {'station': '49002',
  'num': '002',
  'latn': '54.71934167',
  'lone': '19.57249833',
  'datetime': '01-06-2022T17:39:00',
  'date': '01-06-2022',
  'time': '17:39:00',
  'sn': '75748',
  'fname': 'V000213',
  'botdepth': '85.00',
  'odv_datetime': '2022-06-01T17:39:00'},
 '003-49003-V000216-75748': {'station': '49003',
  'num': '003',
  'latn': '54.55723667',
  'lone': '19.56141167',
  'datetime': '01-06-2022T22:51:00',
  'date': '01-06-2022',
  'time': '22:51:00',
  'sn': '75748',
  'fname': 'V000216',
  'botdepth': '58.30',
  'odv_datetime': '2022-06-01T22:51:00'},
 '004-49014-V000224-75748': {'station': '49014',
  'num': '004',
  'latn': '54.86460167',
  'l

In [ ]:
V000073_dbar_CORR V000073_dbar_CORR

In [240]:
print('V000073_dbar_CORR' in '039-49197-V000073_dbar_CORR-43946' and '43946' in '039-49197-V000073_dbar_CORR-43946')

True


In [58]:
m = 'AAaa'
print(m.lower())

aaaa


In [92]:
am = 2

print(f'{am:0>5.8f}')

2.00000000


In [18]:
listoffiles = os.listdir(r'D:\aa_MyExpeditions\Cr_2022-06-01_ABP49\aa_Отчёты\aa_ОтчётОбнинск')

with open(r'D:\aa_MyExpeditions\Cr_2022-06-01_ABP49\aa_Отчёты\aa_ОтчётОбнинск\file.txt', 'w') as f:
    for file in listoffiles:
        f.write(f'{file}\n')